## Question Answering using Deep Learning

In [1]:
import sys
import re
import tarfile
from functools import reduce
from itertools import chain
import os
from six.moves import urllib

import numpy as np
import tensorflow as tf
from tensorflow.python.ops import variable_scope as vs

In [2]:
BABI_DATASET_URL = 'https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz'
DATASET_PATH = './babi'

In [3]:
def fetch_data(dataset_download_url=BABI_DATASET_URL, save_dataset_path=DATASET_PATH): 
    if not os.path.isdir(save_dataset_path):
        os.makedirs(save_dataset_path)
    tar_file_path = os.path.join(save_dataset_path, 'babi_tasks_data_1_20_v1.2.tar.gz')
    urllib.request.urlretrieve(dataset_download_url, tar_file_path)
    babi_tar_file = tarfile.open(tar_file_path)
    babi_tar_file.extractall(path=DATASET_PATH)
    babi_tar_file.close()

In [4]:
fetch_data(dataset_download_url=BABI_DATASET_URL, save_dataset_path=DATASET_PATH)

In [5]:
class Config:
    """Holds model hyperparams and data information.
    Model objects are passed a Config() object at instantiation.
    """

    def __init__(self):
        pass

    batch_size = 32
    embed_size = 50
    hidden_size = 50
    #max_epochs = 5000
    max_epochs = 500
    dropout = 0.7
    lr = 0.8
    L2 = 0.001

    vocab_size = None
    num_steps_sentence = None
    num_steps_story = None
    num_steps_question = None

In [6]:
class RNN_Model:
    def __init__(self, config):
        self.config = config    
        self.add_placeholders()
        self.inputs_story, self.inputs_question = self.add_embedding()
        self.story_question_state = self.add_basic_model(self.inputs_story, self.inputs_question)

        self.output = self.add_projection(self.story_question_state)

        with tf.name_scope('Accuracy'):
            self.predictions = tf.nn.softmax(self.output)
            self.one_hot_prediction = tf.argmax(self.predictions, 1)
            correct_prediction = tf.equal(tf.argmax(self.labels_placeholder, 1), self.one_hot_prediction)
            self.correct_predictions = tf.reduce_sum(tf.cast(correct_prediction, 'int32'))

        with tf.name_scope('Loss'):
            loss, lossL2 = self.add_loss_op(self.output)
            self.calculate_loss = loss + self.config.L2 * lossL2
        with tf.name_scope('Train'):
            self.train_step = self.add_training_op(self.calculate_loss)

    
    def add_placeholders(self):
        """Generate placeholder variables to represent the input tensors
        """
        self.input_story_placeholder = tf.placeholder(
            tf.int32, shape=[None, self.config.num_steps_story, self.config.num_steps_sentence], name='InputStory')
        self.input_question_placeholder = tf.placeholder(
            tf.int32, shape=[None, self.config.num_steps_question], name='InputQuestion')
        self.labels_placeholder = tf.placeholder(
            tf.int32, shape=[None, self.config.vocab_size], name='Target')
        self.X_length = tf.placeholder(tf.int32, shape=[None], name='X_length')
        self.dropout_placeholder = tf.placeholder(tf.float32, name='Dropout')

        
    def add_embedding(self):
        """Add embedding layer.

        Returns:
          inputs: List of length num_steps, each of whose elements should be
                  a tensor of shape (batch_size, embed_size).
        """
        embedding = tf.get_variable('Embedding', [self.config.vocab_size, self.config.embed_size], trainable=True,
                                    initializer=tf.contrib.layers.xavier_initializer())
        inputs_story = tf.nn.embedding_lookup(embedding, self.input_story_placeholder)
        inputs_question = tf.nn.embedding_lookup(embedding, self.input_question_placeholder)

        # Position Encoding
        inputs_question = tf.expand_dims(inputs_question, 1)
        encoded_story = self.get_position_encoding(inputs_story, self.config.num_steps_sentence, 'StoryEncoding')
        encoded_query = self.get_position_encoding(inputs_question, self.config.num_steps_question, 'QueryEncoding')
        encoded_query = tf.tile(encoded_query, tf.stack([1, self.config.num_steps_story, 1]), name=None)

        return encoded_story, encoded_query

    def get_position_encoding(self, embedding, max_length, scope=None):
        """
        Module described in [End-To-End Memory Networks](https://arxiv.org/abs/1502.01852) as Position Encoding (PE).
        The mask allows the ordering of words in a sentence to affect the encoding.
        """
        J, d = max_length, self.config.embed_size
        l = np.zeros((J, d))
        with tf.variable_scope(scope, 'PE'):
            for j in range(J):
                for k in range(d):
                    l[j, k] = (1. - (j + 1.) / J) - ((k + 1.) / d) * (1. - 2. * (j + 1.) / J)
            self.l = tf.constant(l, shape=[J, d])
            m = tf.reduce_sum(embedding * tf.cast(self.l, tf.float32), 2, name='m')
        return m

    def add_projection(self, rnn_output):
        """Adds a projection layer.

        The projection layer transforms the hidden representation to a distribution
        over the vocabulary.

        Args:
          rnn_output: List of length num_steps, each of whose elements should be
                       a tensor of shape (batch_size, embed_size).
        Returns:
          outputs: List of length num_steps, each a tensor of shape
                   (batch_size, len(vocab))
        """
        with tf.variable_scope('Projection'):
            U = tf.get_variable('Weights',
                                [self.config.hidden_size, self.config.vocab_size], trainable=True,
                                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.get_variable('Bias', [self.config.vocab_size])
            outputs = tf.matmul(rnn_output, U) + b

        return outputs

    def add_loss_op(self, output):
        """Adds loss ops to the computational graph.

        Args:
          output: A tensor of shape (None, self.vocab)
        Returns:
          loss: A 0-d tensor (scalar)
        """
        var = tf.trainable_variables()
        cross_entropy = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=self.labels_placeholder))
        tf.add_to_collection('total_loss', cross_entropy)
        loss = tf.add_n(tf.get_collection('total_loss'))
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in var if 'Bias' not in v.name])

        return loss, lossL2

    def add_training_op(self, loss):
        """Sets up the training Ops.
        Args:
          loss: Loss tensor, from cross_entropy_loss.
        Returns:
          train_op: The Op for training.
        """
        optimizer = tf.train.AdagradOptimizer(self.config.lr)
        train_op = optimizer.minimize(loss)

        return train_op
    
    def add_basic_model(self, inputs_story, inputs_question):
        basic_lstm = tf.contrib.rnn.BasicLSTMCell(num_units=self.config.hidden_size)
        basic_lstm = tf.contrib.rnn.DropoutWrapper(basic_lstm, input_keep_prob=self.dropout_placeholder,
                                            output_keep_prob=self.dropout_placeholder)

        with tf.variable_scope('basic_cell') as scope:
            #a, b = basic_dynamic_rnn(basic_lstm, [inputs_story, inputs_question],
            #                                sequence_length=self.X_length,
            #                                dtype=tf.float32)
            a, b = basic_dynamic_rnn(basic_lstm, [inputs_story, inputs_question],
                                            sequence_length=self.X_length,
                                            dtype=tf.float32)

        return tf.reduce_sum(b, 0)
        
    

    def predict(self, session, data):
        input_story, input_question, input_labels, X_length = data
        config = self.config
        dp = 1

        n_data = len(input_story)
        batches = zip(range(0, n_data - config.batch_size, config.batch_size),
                      range(config.batch_size, n_data, config.batch_size))
        batches = [(start, end) for start, end in batches]
        total_correct_examples = 0
        total_processed_examples = 0
        for step, (start, end) in enumerate(batches):
            feed = {self.input_story_placeholder: input_story[start:end],
                    self.input_question_placeholder: input_question[start:end],
                    self.labels_placeholder: input_labels[start:end],
                    self.dropout_placeholder: dp,
                    self.X_length: X_length[start:end]}
            total_correct = session.run(self.correct_predictions, feed_dict=feed)
            total_processed_examples += end - start
            total_correct_examples += total_correct
        acc = total_correct_examples / float(total_processed_examples)

        return acc

    
    def run_epoch(self, session, data, train_op=None, verbose=10):
            input_story, input_question, input_labels, X_length = data
            config = self.config
            dp = config.dropout

            if not train_op:
                train_op = tf.no_op()
                dp = 1

            n_data = len(input_story)
            batches = zip(range(0, n_data - config.batch_size, config.batch_size),
                          range(config.batch_size, n_data, config.batch_size))
            batches = [(start, end) for start, end in batches]
            np.random.shuffle(batches)
            n_val = int(len(batches) * 0.1)
            batches_train = batches[:-n_val]
            batches_val = batches[-n_val:]

            total_loss = []
            total_correct_examples = 0
            total_processed_examples = 0
            total_steps = len(batches_train)
            for step, (start, end) in enumerate(batches_train):
                feed = {self.input_story_placeholder: input_story[start:end],
                        self.input_question_placeholder: input_question[start:end],
                        self.labels_placeholder: input_labels[start:end],
                        self.dropout_placeholder: dp,
                        self.X_length: X_length[start:end]}
                loss, total_correct, _ = session.run(
                    [self.calculate_loss, self.correct_predictions, train_op],
                    feed_dict=feed)
                total_processed_examples += end - start
                total_correct_examples += total_correct
                total_loss.append(loss)
                if verbose and step % verbose == 0:
                    sys.stdout.write('\r{} / {} : loss = {}'.format(
                        step, total_steps, np.mean(total_loss)))
                    sys.stdout.flush()
                if verbose:
                    sys.stdout.write('\r')
            train_acc = total_correct_examples / float(total_processed_examples)

            Story = []
            Question = []
            Answer = []
            Prediction = []
            total_correct_examples = 0
            total_processed_examples = 0
            for step, (start, end) in enumerate(batches_val):
                feed = {self.input_story_placeholder: input_story[start:end],
                        self.input_question_placeholder: input_question[start:end],
                        self.labels_placeholder: input_labels[start:end],
                        self.dropout_placeholder: 1,
                        self.X_length: X_length[start:end]}
                total_correct, prediction = session.run([self.correct_predictions, self.one_hot_prediction], feed_dict=feed)
                total_processed_examples += end - start
                total_correct_examples += total_correct

                Story.append(input_story[start:end])
                Question.append(input_question[start:end])
                Answer.append(input_labels[start:end])
                Prediction.append(prediction)

            val_acc = total_correct_examples / float(total_processed_examples)

            return np.mean(total_loss), train_acc, val_acc, Story, Question, Answer, Prediction
        

In [7]:
def basic_dynamic_rnn(cell_fw, inputs, sequence_length=None,
                                     initial_state_fw=None, 
                                     dtype=None, parallel_iterations=None,
                                     swap_memory=False, time_major=False, scope=None):
    with vs.variable_scope(scope or "unidirectional_rnn"):
        
        with vs.variable_scope("fw") as fw_scope:
            inputs_concat = tf.concat(inputs, 1)
            output_fw, output_state_fw = tf.nn.dynamic_rnn(
                cell=cell_fw, inputs=inputs_concat, sequence_length=sequence_length,
                initial_state=initial_state_fw, dtype=dtype
                )
            
    return output_fw, output_state_fw

## Data Preprocessing

In [8]:
def reset_graph(seed=3242):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


def get_data(task_path):
    train = get_stories(tar.extractfile(task_path.format('train')))
    test = get_stories(tar.extractfile(task_path.format('test')))
    return train, test

def get_stories(f, only_supporting=False, max_length=None):
    """Given a file name, read the file, retrieve the stories, and then convert the sentences into a single story.
    If max_length is supplied, any stories longer than max_length tokens will be discarded.
    """
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    data = [(story, q, answer) for story, q, answer in data if
            not max_length or len(story) < max_length]
    return data
    

def parse_stories(lines, only_supporting=False):
    """
    Parse the bAbI task format.
    If only_supporting is True, only the sentences that support the answer are kept.
    """
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize_word(q)
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize_word(line)
            story.append(sent)
    return data


def tokenize_word(sent):
    """Return the tokens of a sentence excluding punctuation.
    >> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', 'Bob', 'went', 'to', 'the', 'kitchen']
    """
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def vectorize_stories(data, word_idx, sentence_maxlen, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    X_length = []

    for story, query, answer in data:
        sentences = []
        for s in story:
            sentence = [word_idx[w] for w in s]
            for _ in range(sentence_maxlen - len(sentence)):
                sentence.append(0)
            assert len(sentence) == sentence_maxlen
            sentences.append(sentence)
        X_length.append(len(sentences))

        ## story
        for _ in range(story_maxlen - len(sentences)):
            sentences.append([0 for _ in range(sentence_maxlen)])

        ## query
        xq = [word_idx[w] for w in query]
        for _ in range(query_maxlen - len(xq)):
            xq.append(0)
        ## answer
        y = np.zeros(len(word_idx) + 1)  # index 0 is reserved
        y[word_idx[answer]] = 1

        X.append(sentences)
        Xq.append(xq)
        Y.append(y)

    return X, Xq, np.array(Y), X_length


In [ ]:
reset_graph()
tasks = [
    'qa1_single-supporting-fact', 'qa2_two-supporting-facts', 'qa3_three-supporting-facts',
    'qa4_two-arg-relations', 'qa5_three-arg-relations', 'qa6_yes-no-questions', 'qa7_counting',
    'qa8_lists-sets', 'qa9_simple-negation', 'qa10_indefinite-knowledge',
    'qa11_basic-coreference', 'qa12_conjunction', 'qa13_compound-coreference',
    'qa14_time-reasoning', 'qa15_basic-deduction', 'qa16_basic-induction', 'qa17_positional-reasoning',
    'qa18_size-reasoning', 'qa19_path-finding', 'qa20_agents-motivations'
]

verbose = True
path = 'babi/babi_tasks_data_1_20_v1.2.tar.gz'
tar = tarfile.open(path)
tasks_dir = 'tasks_1-20_v1-2/en/'


for task in tasks:
    print('\n**************************************\n')
    print(task)
    
    task_path = tasks_dir + task + '_{}.txt'
    train, test = get_data(task_path)
    
    flatten = lambda data: [i for i in chain(*data)]
    vocab = sorted(reduce(lambda x, y: x | y, (set(flatten(story) + q + [answer])
                                               for story, q, answer in train + test)))

    # 0 is reserved for masking via pad_sequences
    vocab_size = len(vocab) + 1
    word_idx = dict((c, i + 1) for i, c in enumerate(vocab))

    sentence_maxlen = max(flatten([[len(s) for s in x] for x, _, _ in train + test]))
    #sentence_maxlen = max([i for i in chain(*[[len(s) for s in x] for x, _, _ in train + test])])
    story_maxlen = max(map(len, (x for x, _, _ in train + test)))
    query_maxlen = max(map(len, (x for _, x, _ in train + test)))
    
    idx_word = {v: k for k, v in word_idx.iteritems()}
    idx_word[0] = "_PAD"
     
    X, Xq, Y, X_length = vectorize_stories(train, word_idx, sentence_maxlen, story_maxlen, query_maxlen)
    tX, tXq, tY, tX_length = vectorize_stories(test, word_idx, sentence_maxlen, story_maxlen, query_maxlen)
    
    if verbose:
        print('vocab = {}'.format(vocab))
        print('word_idx = {}'.format(word_idx))
        print('X.shape = {}'.format(np.array(X).shape))
        print('Xq.shape = {}'.format(np.array(Xq).shape))
        print('Y.shape = {}'.format(Y.shape))
        print('story_maxlen, sentence_maxlen, query_maxlen = {}, {}, {}'.format(story_maxlen, sentence_maxlen, query_maxlen))

    config = Config()
    config.vocab_size = vocab_size
    config.num_steps_sentence = sentence_maxlen
    config.num_steps_story = story_maxlen
    config.num_steps_question = query_maxlen
    print('max_epochs = {}'.format(config.max_epochs)) 
    with tf.Graph().as_default() as g:
        model = RNN_Model(config)
        init = tf.global_variables_initializer()

        with tf.Session() as session:
            session.run(init)

            for epoch in range(config.max_epochs):
                print('Epoch {}'.format(epoch))

                train_loss, train_acc, val_acc, Story, Question, Answer, Prediction = model.run_epoch(session, (
                    X, Xq, Y, X_length), train_op=model.train_step)

                if verbose:
                    print('Training loss: {}'.format(train_loss))
                    print('Training acc: {}'.format(train_acc))
                    print('Validation acc: {}'.format(val_acc))


                if epoch % 20 == 0:
                    test_acc = model.predict(session, (tX, tXq, tY, tX_length))
                    print('Testing acc: {}'.format(test_acc))


**************************************

qa1_single-supporting-fact
vocab = [u'.', u'?', u'Daniel', u'John', u'Mary', u'Sandra', u'Where', u'back', u'bathroom', u'bedroom', u'garden', u'hallway', u'is', u'journeyed', u'kitchen', u'moved', u'office', u'the', u'to', u'travelled', u'went']
word_idx = {u'hallway': 12, u'bathroom': 9, u'garden': 11, u'journeyed': 14, u'office': 17, u'is': 13, u'bedroom': 10, u'moved': 16, u'back': 8, u'went': 21, u'.': 1, u'to': 19, u'travelled': 20, u'Daniel': 3, u'Sandra': 6, u'the': 18, u'John': 4, u'Where': 7, u'Mary': 5, u'?': 2, u'kitchen': 15}
X.shape = (1000, 10, 7)
Xq.shape = (1000, 4)
Y.shape = (1000, 22)
story_maxlen, sentence_maxlen, query_maxlen = 10, 7, 4
max_epochs = 500
Epoch 0
Training loss: 21.600284576485
Training acc: 0.142857142857
Validation acc: 0.125
Testing acc: 0.193548387097
Epoch 1
Training loss: 6.8392348289587
Training acc: 0.15625
Validation acc: 0.229166666667
Epoch 2
20 / 28 : loss = 5.76768684387Training loss: 5.55673456192